<a href="https://colab.research.google.com/github/mattmason36/pyTorch-testing/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [3]:
# Convert MNIST image files into tensor of 4 dimensions (# of images, height,
# width, color channel)
transform = transforms.ToTensor()

In [4]:
# train data
train_data = datasets.MNIST(root='/cnn_data', train = True, download = True, transform = transform)

100%|██████████| 9.91M/9.91M [00:01<00:00, 4.97MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 160kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.67MB/s]


In [5]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train = False, download = True, transform = transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [8]:
# Create a small batch size for images....lets say 10
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

In [9]:
# Define our CNN Model
# Decribe convolutional layer and what it`s doing (2 convolutional layers)
# This is just an example in the next video we`ll build out the actual model
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [10]:
# Grab one MNIST record
for i, (X_Train, y_train) in enumerate(train_data):
    break

In [11]:
X_Train.shape

torch.Size([1, 28, 28])

In [12]:
x = X_Train.view(1, 1, 28, 28)

In [13]:
# Perform our first convolution
x = F.relu(conv1(x)) # Recrified Linear Unit for our activation function

In [14]:
# 1 single image, 6 is the filters we asked for, 26x26
x.shape

torch.Size([1, 6, 26, 26])

In [15]:
# pass thru the pooling layer
x= F.max_pool2d(x, 2, 2) # kernel of 2 and stride of 2

In [16]:
x.shape # 26/2 = 13

torch.Size([1, 6, 13, 13])

In [17]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [18]:
x.shape # Agaion we didnt set padding so we loose the 2 pixels of the outline of the image

torch.Size([1, 16, 11, 11])

In [19]:
# Pooling layer
x = F.max_pool2d(x, 2, 2)

In [20]:
x.shape # 11/2 = 5.5 we have to round down, because we can't invent data to round up

torch.Size([1, 16, 5, 5])

In [21]:
((28-2)/2-2)/2

5.5

In [22]:
# Model class
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        # Fully connected
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2) # 2x2 kernel and stride 2
        # Second pass
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2) # 2x2 kernel and stride 2
        # Flatten
        X = X.view(-1, 16*5*5) # Negative one ss that we can vary the batch size

        # Fully connected layers
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim = 1)

In [23]:
# Create an instance of our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [24]:
# Loss function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # Smaller the learning rate the longer its gonna take to train

In [30]:
import time
start_time = time.time()

# Create vaialbles to track tings
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
    trn_corr = 0
    tst_corr = 0


    # Train
    for b, (X_train, y_train) in enumerate(train_loader):
        b+=1 # start our batches at 1
        y_pred = model(X_train) # get predicted values from the training set. Not flattened
        loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

        predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point
        batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False = 0, sum those up
        trn_corr += batch_corr # keep track as we go along in training

        # Update our parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        # print out some results
        if b % 600 == 0:
            print(f'epoch: {i} batch: {b} loss: {loss.item()}')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    # Test

    with torch.no_grad(): # No gradients so we dont update our weights and biases with data
        for b, (X_test, y_test) in enumerate(test_loader):
            y_val = model(X_test)
            predicted = torch.max(y_val.data, 1)[1] # Add up correct predictions
            tst_corr += (predicted == y_test).sum() # T=1, F=0 and sum away

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)


current_time = time.time()
total = current_time - start_time
print(f'Training took: {total/60} minutes!')

epoch: 0 batch: 600 loss: 0.0003118058084510267
epoch: 0 batch: 1200 loss: 0.001414088299497962
epoch: 0 batch: 1800 loss: 0.00021644712251145393
epoch: 0 batch: 2400 loss: 0.05918513983488083
epoch: 0 batch: 3000 loss: 0.0007876444724388421
epoch: 0 batch: 3600 loss: 0.0006503040203824639
epoch: 0 batch: 4200 loss: 0.0006699328077957034
epoch: 0 batch: 4800 loss: 7.764544716337696e-05
epoch: 0 batch: 5400 loss: 0.0006076421122997999
epoch: 0 batch: 6000 loss: 0.0015855187084525824
epoch: 1 batch: 600 loss: 1.2420893654052634e-05
epoch: 1 batch: 1200 loss: 0.10316796600818634
epoch: 1 batch: 1800 loss: 0.0004227778990752995
epoch: 1 batch: 2400 loss: 6.861152360215783e-05
epoch: 1 batch: 3000 loss: 7.363016629824415e-05
epoch: 1 batch: 3600 loss: 0.02122342959046364
epoch: 1 batch: 4200 loss: 0.009393647313117981
epoch: 1 batch: 4800 loss: 5.7252589613199234e-05
epoch: 1 batch: 5400 loss: 0.03745017200708389
epoch: 1 batch: 6000 loss: 0.9072400331497192
epoch: 2 batch: 600 loss: 0.0024